In [5]:
# tester.ipynb
import sys
from pathlib import Path

PROJECT_ROOT = Path.cwd().parent
sys.path.insert(0, str(PROJECT_ROOT))


from src.analyzer import analyze_speech

# Quick analysis
result = analyze_speech("../samples/sample4.flac")

# Access results
print(f"Score: {result['verdict']['fluency_score']}")
print(f"Fillers detected: {len(result['filler_events'])}")

Analyzing audio: ../samples/sample4.flac
Context: conversational

[1/6] Transcribing with Whisper...
  Duration: 23.08s
  Words: 30

[2/6] Aligning words with WhisperX...
  Aligned: 30 words

[3/6] Detecting fillers with Wav2Vec2...


Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  Detected: 30 events

[4/6] Detecting fillers with Whisper (verbatim)...
  Detected: 5 fillers

[5/6] Merging detections...
  Total events: 14

[6/6] Calculating fluency score...

✓ Fluency Score: 62/100
✓ Readiness: borderline
Score: 62
Fillers detected: 14


In [4]:
result

{'verdict': {'fluency_score': 62, 'readiness': 'borderline'},
 'benchmarking': {'percentile': 30,
  'target_score': 80,
  'score_gap': 18,
  'estimated_guided_practice_hours': 10.799999999999999},
 'normalized_metrics': {'wpm': 77.98960138648181,
  'fillers_per_min': np.float64(9.200000000000001),
  'stutters_per_min': 10.0,
  'long_pauses_per_min': 0.0,
  'very_long_pauses_per_min': 0.0,
  'pause_time_ratio': np.float64(0.05199306759098784),
  'pause_variability': 0.0,
  'vocab_richness': 0.8333333333333334,
  'repetition_ratio': np.float64(0.06666666666666667)},
 'opinions': {'primary_issues': [{'issue': 'filler_dependency',
    'severity': 'high',
    'root_cause': 'Fillers replace silent planning pauses.',
    'score_impact': 25},
   {'issue': 'delivery_pacing',
    'severity': 'medium',
    'root_cause': 'Speech rate deviates from optimal clarity range.',
    'score_impact': 12}],
  'action_plan': [{'priority': 1,
    'focus': 'filler_dependency',
    'instruction': 'Replace fille